In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from keras.utils.vis_utils import plot_model
from itertools import chain
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import os
import time
from itertools import chain
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16115166817913871350
]


In [3]:
import pandas as pd

df = pd.read_json('../datasets/7_dataset_SM.json')
df = df[:5000]
df = df.fillna(method='ffill')

In [5]:
df

,Sentence #,Word,Lemma,POS,Dep,Polarity,Label
0,1,At,at,ADV,advmod,0.000000,W
1,1,least,least,ADV,advmod,0.000000,W
2,1,two,two,NUM,nummod,0.000000,W
3,1,organizations,organization,NOUN,nsubj,-0.193345,W
4,1,have,have,AUX,aux,-0.046438,W
...,...,...,...,...,...,...,...
4995,235,not,not,PART,neg,-0.002688,M
4996,235,signed,sign,VERB,relcl,-0.096119,M
4997,235,by,by,ADP,agent,0.000000,M
4998,235,an,an,DET,det,0.000000,M


In [14]:
#Create mapper for features

lemma_vocab = np.unique(df['Lemma']).tolist()
POS_vocab = np.unique(df['POS']).tolist()
# dep_vocab = np.unique(df['Dep']).tolist()

i_to_lemma = {i:lemma for  i, lemma in enumerate(lemma_vocab)}
lemma_to_i = {lemma:i for  i, lemma in enumerate(lemma_vocab)}

i_to_POS = {i:POS for  i, POS in enumerate(POS_vocab)}
POS_to_i = {POS:i for  i, POS in enumerate(POS_vocab)}

In [32]:
#Append index columns for lemmas and POS and group dataframe by sentence

df['Lemma_index'] = df['Lemma'].map(lemma_to_i)
df['POS_index'] = df['POS'].map(POS_to_i)

df_grouped = df.groupby(['Sentence #'],as_index=False)['Word', 'Lemma', 'POS', 'Dep', 'Polarity', 'Lemma_index', 'POS_index', 'Label'].agg(lambda x: list(x))

C:\Users\hanse\AppData\Local\Temp/ipykernel_5180/1779994312.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_grouped = df.groupby(['Sentence #'],as_index=False)['Word', 'Lemma', 'POS', 'Dep', 'Polarity', 'Lemma_index', 'POS_index', 'Label'].agg(lambda x: list(x))


In [33]:
df_grouped

,Sentence #,Word,Lemma,POS,Dep,Polarity,Lemma_index,POS_index,Label
0,1,"[At, least, two, organizations, have, decided,...","[at, least, two, organization, have, decide, t...","[ADV, ADV, NUM, NOUN, AUX, VERB, PART, VERB, P...","[advmod, advmod, nummod, nsubj, aux, ROOT, aux...","[0.0, 0.0, 0.0, -0.19334491413051003, -0.04643...","[373, 789, 1239, 915, 681, 528, 1214, 561, 200...","[2, 2, 8, 7, 3, 16, 9, 16, 11, 11, 1, 8, 7, 16...","[W, W, W, W, W, W, W, W, W, W, W, W, W, W, W, ..."
1,2,"[Women, who, previously, worked, with, the, Os...","[woman, who, previously, work, with, the, Osca...","[NOUN, PRON, ADV, VERB, ADP, DET, PROPN, PUNCT...","[nsubj, nsubj, advmod, relcl, prep, det, npadv...","[0.35316112487835105, 0.0, -0.0393333114017160...","[1307, 1298, 990, 1308, 1304, 1194, 221, 11, 1...","[7, 10, 2, 16, 1, 5, 11, 12, 16, 7, 16, 11, 13...","[W, W, W, W, W, W, W, W, W, W, W, W, W, W, W, ..."
2,3,"[In, response, to, the, allegations, ,, Visa, ...","[in, response, to, the, allegation, ,, Visa, a...","[ADP, NOUN, ADP, DET, NOUN, PUNCT, PROPN, VERB...","[prep, pobj, prep, det, pobj, punct, nsubj, RO...","[-0.005527309776069, -0.039620517765398, -0.0,...","[719, 1064, 1214, 1194, 332, 10, 278, 347, 755...","[1, 7, 1, 5, 7, 12, 11, 16, 10, 3, 16, 10, 1, ...","[W, W, W, W, W, W, W, W, W, W, W, W, W, W, W, ..."
3,4,"[ , We, are, aware, of, the, allegations, that...","[ , we, be, aware, of, the, allegation, that, ...","[SPACE, PRON, AUX, ADJ, ADP, DET, NOUN, DET, A...","[punct, nsubj, ROOT, acomp, prep, det, pobj, n...","[0.0, -0.0, 0.0, -0.035281092794016006, 0.0, 0...","[0, 1284, 393, 385, 899, 1194, 332, 1193, 681,...","[14, 10, 3, 0, 1, 5, 7, 5, 3, 3, 16, 1, 11, 11...","[W, W, W, W, W, W, W, W, W, W, W, W, W, W, W]"
4,5,"[At, this, point, ,, Visa, will, be, suspendin...","[at, this, point, ,, Visa, will, be, suspend, ...","[ADP, DET, NOUN, PUNCT, PROPN, AUX, AUX, VERB,...","[prep, det, pobj, punct, nsubj, aux, aux, ccom...","[0.0, 0.0, -0.19485058785637502, 0.0, -0.02323...","[373, 1204, 962, 10, 278, 1299, 393, 1179, 918...","[1, 5, 7, 12, 11, 3, 3, 16, 10, 7, 1, 5, 5, 7,...","[W, W, W, W, W, W, W, W, W, W, W, W, W, W, W, ..."
...,...,...,...,...,...,...,...,...,...
230,231,"[We, will, continue, the, conversation, this, ...","[we, will, continue, the, conversation, this, ...","[PRON, AUX, VERB, DET, NOUN, DET, NOUN, PUNCT]","[nsubj, aux, ROOT, det, dobj, det, npadvmod, p...","[-0.0, -0.005636411889281, -0.1172245756221810...","[1284, 1299, 496, 1194, 499, 1204, 855, 12]","[10, 3, 16, 5, 7, 5, 7, 12]","[M, M, M, M, M, M, M, M]"
231,232,[ ],[ ],[SPACE],[ROOT],[0.0],[0],[14],[M]
232,233,"[The, league, also, debated, the, issue, in, p...","[the, league, also, debate, the, issue, in, pr...","[DET, NOUN, ADV, VERB, DET, NOUN, ADP, ADJ, NO...","[det, nsubj, advmod, ROOT, det, dobj, prep, am...","[0.0, -0.18007909868621702, 0.0, -0.0284715738...","[1194, 787, 338, 526, 1194, 754, 719, 989, 839...","[5, 7, 2, 16, 5, 7, 1, 0, 7, 13, 0, 11, 7, 11,...","[M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, ..."
233,234,"[The, movement, ,, which, grew, to, include, o...","[the, movement, ,, which, grow, to, include, o...","[DET, NOUN, PUNCT, DET, VERB, PART, VERB, ADJ,...","[det, nsubj, punct, nsubj, relcl, aux, advcl, ...","[0.0, 0.011468645017082, 0.0, 0.0, -0.12359977...","[1194, 861, 10, 1295, 671, 1214, 723, 917, 957...","[5, 7, 12, 5, 16, 9, 16, 0, 7, 4, 7, 12, 16, 1...","[M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, ..."


In [27]:
max_POS = len(POS_vocab)
max_lemma = len(lemma_vocab)

tokens = df_grouped['Lemma_index'].tolist()
POS = df_grouped['POS_index'].tolist()

max_length = max([len(s) for s in tokens])

pad_tokens = pad_sequences(tokens, maxlen=max_length, dtype='int32', padding='post', value= max_lemma - 1)

pad_POS = pad_sequences(POS, maxlen=max_length, dtype='int32', padding='post', value= POS_to_i["X"])
n_POS = len(POS_to_i)

pad_POS = [to_categorical(i, num_classes=n_POS) for i in pad_POS]

pad_labels = pad_sequences(tokens, maxlen=max_length, dtype='int32', padding='post', value= max_lemma - 1)

tokens_, test_tokens, POS_, test_POS = train_test_split(pad_tokens, pad_POS, test_size=None, random_state=0)
train_tokens, test_tokens, train_POS, test_POS = train_test_split(tokens_,POS_,test_size = 0.3, random_state=0)

In [28]:
input_dim = len(lemma_vocab)+1
output_dim = 64
input_length = max([len(s) for s in df_grouped['Lemma_index'].tolist()])
n_POS = len(POS_vocab)

In [29]:
root_logdir = os.path.join(os.curdir, "la_logs")
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_log_dir = get_run_logdir()
run_log_dir

tensorboard_cb = tf.keras.callbacks.TensorBoard(run_log_dir)

In [30]:
model = tf.keras.models.Sequential(
    [
        layers.Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length), # Embedding vs Input layer
        layers.Bidirectional(layers.LSTM(units=output_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat'),
        layers.LSTM(units=output_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.5),
        layers.TimeDistributed(layers.Dense(n_POS, activation="softmax")) #Use softmax for activation HOML page 383
    ]
)

adam = tf.keras.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999)

#Hyper params to change - learning rate, number of layers and neurons, activations functions

model.compile(loss='crossentropy', optimizer='adam', metrics=['accuracy']) # cross entropy loss chapter 4 HOML - categorial crossentropy because to_categorial 
model.summary()
plot_model(model)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1176, 64)          85184     
_________________________________________________________________
bidirectional (Bidirectional (None, 1176, 128)         66048     
_________________________________________________________________
lstm_1 (LSTM)                (None, 1176, 64)          49408     
_________________________________________________________________
time_distributed (TimeDistri (None, 1176, 18)          1170      
Total params: 201,810
Trainable params: 201,810
Non-trainable params: 0
_________________________________________________________________
('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


C:\Users\hanse\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [31]:
history = model.fit(train_tokens, np.array(train_POS), epochs=20, callbacks=[tensorboard_cb], verbose=2)
%load_ext tensorboard
%tensorboard --logdir=./la_logs --port=6006

Epoch 1/20
1/1 - 11s - loss: 2.8833 - accuracy: 0.0325
Epoch 2/20
1/1 - 12s - loss: 2.8450 - accuracy: 0.5638
Epoch 3/20
1/1 - 2s - loss: 2.8040 - accuracy: 0.5403
Epoch 4/20
1/1 - 2s - loss: 2.7569 - accuracy: 0.5316
Epoch 5/20
1/1 - 2s - loss: 2.6957 - accuracy: 0.5304
Epoch 6/20
1/1 - 2s - loss: 2.6108 - accuracy: 0.5304
Epoch 7/20
1/1 - 2s - loss: 2.4965 - accuracy: 0.5304
Epoch 8/20
1/1 - 2s - loss: 2.3332 - accuracy: 0.5304
Epoch 9/20
1/1 - 2s - loss: 2.1281 - accuracy: 0.5304
Epoch 10/20
1/1 - 2s - loss: 1.9001 - accuracy: 0.5304
Epoch 11/20
1/1 - 2s - loss: 1.7091 - accuracy: 0.5304
Epoch 12/20
1/1 - 2s - loss: 1.6014 - accuracy: 0.5304
Epoch 13/20
1/1 - 2s - loss: 1.5774 - accuracy: 0.5304
Epoch 14/20
1/1 - 2s - loss: 1.5683 - accuracy: 0.5304
Epoch 15/20
1/1 - 2s - loss: 1.5023 - accuracy: 0.5304
Epoch 16/20
1/1 - 2s - loss: 1.4346 - accuracy: 0.5310
Epoch 17/20
1/1 - 2s - loss: 1.4068 - accuracy: 0.5679
Epoch 18/20
1/1 - 2s - loss: 1.3945 - accuracy: 0.6010
Epoch 19/20
1/1 -

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.